In [1]:
from utils import *
from dataset  import *
from models import *
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/anjadhav/miniconda3/envs/chemIR/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_file = "gtest_iob.csv"

In [3]:
pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"
batch_size = 8
max_para_length = 128
para_seq_len = 16  #number of paras to be encoded and decoded together (hyperparameter)
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

# Make sure you choose suitable num_worker, otherwise it will result in errors
num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  8  system version =  3.7.13 (default, Oct 18 2022, 18:57:03) 
[GCC 11.2.0]


In [4]:
test_data = CRFEmbeddingDataset(test_file, para_seq_len = para_seq_len, pretrained_model = pretrained_model, stride = para_seq_len)
test_args = dict(shuffle=False, batch_size=batch_size, num_workers=8, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_data, **test_args)


128


In [5]:

print(test_data.__len__())
print(len(test_loader))

826
104


## Model 1
./chem_bert_iob_bilstm_crf_bert_finetune/model_model_params_0.9507615640930901.pth

pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"

batch_size = 4

max_para_length = 128

para_seq_len = 16  #number of paras to be encoded and decoded together (hyperparameter)


In [6]:
model = EncoderDecoderBiLstmCRF(embed_model = BertEmbedding(pretrained_model), num_tags = 3, freeze_bert=False)
model.load_state_dict(torch.load('./chem_bert_iob_bilstm_crf_bert_finetune/model_model_params_0.9507615640930901.pth'))
# model = load_pretrained_weights(model, './model_model_params_0.9428545098368426.pth')

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   model = nn.DataParallel(model)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_total_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print("Total params: ", total_params)
print("Trainable params: ", trainable_total_params)
print("Non Trainable params: ", non_trainable_total_params)


Some weights of the model checkpoint at recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [ ]:
test_df = pd.read_csv(test_file)
test_predictions = extract(model, test_loader, device)
test_df, pred_spans = get_spans(test_df, test_predictions)

learning_rate: [5e-05]
Validation loss: 5.04, Time: 29.1264865398407
{'eval_accuracy': 0.9406094527363185, 'eval_precision': 0.9394594020800763, 'eval_recall': 0.9406094527363185, 'eval_f-1': 0.9397341370157223}
Total original spans:  1234
Total predicted spans:  1104
Total number of original spans correctly predicted acc to strict match:  854
Percent of original spans correctly predicted acc to strict match:  69.20583468395462
Total number of original spans correctly predicted acc to fuzzy match:  962
Percent of original spans correctly predicted acc to fuzzy match:  77.9578606158833
Count of fuzzy matched spans:  108
Count of spans with misaligned begin and end: 13 (12.04%) 
Count of spans with misaligned begin: 26 (24.07%) 
Count of spans with misaligned end: 69 (63.89%) 


In [ ]:
test_df.to_csv("gtest_predictions.csv")